In [37]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [38]:
%cd gdrive/My Drive/Colab Notebooks/DerinOgrenme/
%ls

[Errno 2] No such file or directory: 'gdrive/My Drive/Colab Notebooks/DerinOgrenme/'
/content/gdrive/My Drive/Colab Notebooks/DerinOgrenme
deneme.jpg  MobileNet61.h5  VGG16_1.h5  VGG19_2.h5  VGG19_4.h5
kekik.jpg   test/           VGG19_1.h5  VGG19_3.h5  VGG19_5.h5


In [41]:

from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras import backend as K
from PIL import Image
import numpy as np 
import pandas as pd 
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet




img_width, img_height = 224, 224

imageChannels = 3
batchSize = 100
epoch = 20
classMode = 'categorical'
fcDense = 3
fcActivation = 'softmax' #son katman
optimizer = 'adam'
loss = 'binary_crossentropy'
metrics = 'accuracy'
train_ornek_sayisi = 3000
validation_ornek_sayisi = 900 


train_data_yolu = 'test/train'
validation_data_yolu = 'test/validation'


if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [42]:
input = Input(input_shape)

model = MobileNet( weights='imagenet',
                  include_top=False,
                  input_shape=input_shape,
                  input_tensor=input
                  )

In [44]:
flat1 = Flatten()(model.layers[-1].output)

class1=Dense(128,activation='relu')(flat1)
class2=Dense(256,activation='relu')(class1) 
class3=Dense(256,activation='relu')(class2) 

output=Dense(3,activation='softmax', name='predictions')(class3) 

In [45]:
model=Model(inputs=model.inputs,outputs=output)

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])


model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                                    

In [47]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2, horizontal_flip=True, zoom_range=0.2, rescale=1./255, rotation_range=40, width_shift_range=0.2,
                                 height_shift_range=0.2, shear_range=0.2, fill_mode='nearest')
test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    train_data_yolu,
    target_size=(img_width, img_height),
    batch_size=batchSize,
    subset="training",
    class_mode=classMode
)
validation_generator = train_datagen.flow_from_directory(
    train_data_yolu,
    target_size=(img_width, img_height),
    batch_size=batchSize,
    subset="validation",
    class_mode=classMode
)
test_generator = test_datagen.flow_from_directory(
    validation_data_yolu,
    target_size=(img_width, img_height),
    batch_size=batchSize,
    class_mode=classMode
)

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.
Found 900 images belonging to 3 classes.


In [48]:
history=model.fit(
                  train_generator,
                  validation_data=validation_generator,
                  steps_per_epoch=2400//batchSize,                      
                  validation_steps=600//batchSize,
                  batch_size=480,
                  epochs=epoch,
                  verbose=1
  )

Epoch 1/20
24/24 [==============================] - 53s 2s/step - loss: 1.3854 - accuracy: 0.8392 - val_loss: 14.4580 - val_accuracy: 0.3333
Epoch 2/20
24/24 [==============================] - 49s 2s/step - loss: 0.0631 - accuracy: 0.9837 - val_loss: 6.7395 - val_accuracy: 0.3333
Epoch 3/20
24/24 [==============================] - 49s 2s/step - loss: 0.0103 - accuracy: 0.9962 - val_loss: 2.4917 - val_accuracy: 0.3333
Epoch 4/20
24/24 [==============================] - 49s 2s/step - loss: 0.0054 - accuracy: 0.9983 - val_loss: 3.9448 - val_accuracy: 0.3333
Epoch 5/20
24/24 [==============================] - 49s 2s/step - loss: 0.0136 - accuracy: 0.9971 - val_loss: 5.3182 - val_accuracy: 0.3333
Epoch 6/20
24/24 [==============================] - 49s 2s/step - loss: 0.0206 - accuracy: 0.9958 - val_loss: 4.6464 - val_accuracy: 0.1667
Epoch 7/20
24/24 [==============================] - 50s 2s/step - loss: 0.0082 - accuracy: 0.9983 - val_loss: 6.3114 - val_accuracy: 0.3333
Epoch 8/20
24/24 [=

In [53]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

img = load_img('kekik.jpg', grayscale=False, color_mode='rgb', target_size=(img_width, img_height))
image = img_to_array(img)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image = preprocess_input(image)

yhat = model.predict(image)
print(yhat)


[[7.3246253e-14 5.1570006e-13 1.0000000e+00]]


In [54]:
model.save("MobileNet73.h5")